In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [40]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='/Users/woosh/Study/CPE393 MLops/cpe393-mlflow/mlruns/2', creation_time=1743015352823, experiment_id='2', last_update_time=1743015352823, lifecycle_stage='active', name='lab-nyc-green-taxi-model', tags={}>,
 <Experiment: artifact_location='/Users/woosh/Study/CPE393 MLops/cpe393-mlflow/mlruns/1', creation_time=1743014891800, experiment_id='1', last_update_time=1743014891800, lifecycle_stage='active', name='mlops_nyc_taxi', tags={}>,
 <Experiment: artifact_location='/Users/woosh/Study/CPE393 MLops/cpe393-mlflow/mlruns/0', creation_time=1743014891799, experiment_id='0', last_update_time=1743014891799, lifecycle_stage='active', name='Default', tags={}>]

In [3]:
client.create_experiment(name="lab-nyc-green-taxi-model")

'2'

In [44]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 100",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [45]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 52743c9556ad46eabf3d05ce48cfae90, rmse: 26.0516
run id: 9e9dd776e8134358a86a141d18cef975, rmse: 26.7294
run id: 8bebe909c7f544f2bb2d6646d48127a3, rmse: 27.4219
run id: e24053e7eb244f9699437379f9912b9c, rmse: 28.6579
run id: a2aab96616b84dd19b67c630250b16c5, rmse: 31.1900


In [46]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [47]:
run_id = "2ebe874f58ed4fe58cd939dd0ecb2b08"  #insert your run id
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '2' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1743017821012, current_stage='None', description=None, last_updated_timestamp=1743017821012, name='nyc-taxi-regressor', run_id='2ebe874f58ed4fe58cd939dd0ecb2b08', run_link=None, source=('/Users/woosh/Study/CPE393 '
 'MLops/cpe393-mlflow/mlruns/1/2ebe874f58ed4fe58cd939dd0ecb2b08/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=2>

In [54]:
#check model version and stages
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Production
version: 2, stage: Staging


/var/folders/9p/zg1k26nd61j1lr7bdl1cn2100000gn/T/ipykernel_68358/1279244624.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [53]:
model_version = 2
new_stage = "Staging"
# Use client.transition_model_version_stage() function to model to staging.
client.transition_model_version_stage(model_name,model_version,new_stage)

/var/folders/9p/zg1k26nd61j1lr7bdl1cn2100000gn/T/ipykernel_68358/3812018123.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(model_name,model_version,new_stage)


<ModelVersion: aliases=[], creation_timestamp=1743017821012, current_stage='Staging', description=None, last_updated_timestamp=1743017891692, name='nyc-taxi-regressor', run_id='2ebe874f58ed4fe58cd939dd0ecb2b08', run_link=None, source=('/Users/woosh/Study/CPE393 '
 'MLops/cpe393-mlflow/mlruns/1/2ebe874f58ed4fe58cd939dd0ecb2b08/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=2>

Comparing versions and selecting the new "Production" model
In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
Preprocess the test set using the DictVectorizer so we can properly feed the regressors.
Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
Based on the results, update the "Production" model version accordingly.


In [50]:
import pandas as pd

df = pd.read_csv("https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-03.csv.gz", compression='gzip')

df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2019-03-01 00:02:21,2019-03-01 00:04:52,N,1,193,193,1,0.00,3.5,0.5,0.5,1.44,0.0,NaN,0.3,6.24,1,1,0.0
1,2,2019-03-01 00:42:31,2019-03-01 00:47:42,N,1,80,256,1,0.72,5.5,0.5,0.5,1.36,0.0,NaN,0.3,8.16,1,1,0.0
2,1,2019-03-01 00:15:53,2019-03-01 00:18:37,N,1,7,179,1,0.50,4.0,0.5,0.5,0.00,0.0,NaN,0.3,5.30,2,1,0.0
3,2,2019-02-28 23:55:58,2019-03-01 00:01:30,N,1,256,80,1,1.19,6.0,0.5,0.5,1.46,0.0,NaN,0.3,8.76,1,1,0.0
4,2,2019-03-01 00:55:58,2019-03-01 00:58:12,N,1,223,223,1,0.55,4.0,0.5,0.5,0.00,0.0,NaN,0.3,5.30,2,1,0.0


In [55]:
import mlflow
import pickle
from mlflow.artifacts import download_artifacts

# Download DictVectorizer from MLflow artifacts
artifact_path = f"runs:/2ebe874f58ed4fe58cd939dd0ecb2b08/preprocessor/preprocessor.b"

try:
    vectorizer_local_path = mlflow.artifacts.download_artifacts(artifact_path)
except Exception as e:
    print(f"An error occurred while downloading the artifact: {e}")
    vectorizer_local_path = None

# Load the DictVectorizer
with open(vectorizer_local_path, "rb") as f:
    dv = pickle.load(f)

# Transform the dataset
dicts = df.to_dict(orient="records")
X_test = dv.transform(dicts)

print("DictVectorizer successfully loaded and applied to test data.")


DictVectorizer successfully loaded and applied to test data.


In [59]:
from mlflow.pyfunc import load_model
from sklearn.metrics import mean_squared_error
import numpy as np

staging_model_path = f"runs:/2ebe874f58ed4fe58cd939dd0ecb2b08/models_mlflow"
# Load the Staging and Production models
staging_model = load_model(staging_model_path)
production_model = client.get_latest_versions(model_name, stages=["Production"])
if production_model:
    production_version = production_model[0].version
    production_model = load_model(f"runs:/dc12ee5f06bb4bb7a8304c7d4f11fbb4/models_mlflow")
else:
    production_model = None  # No production model yet
    print("None")


/var/folders/9p/zg1k26nd61j1lr7bdl1cn2100000gn/T/ipykernel_68358/1945465347.py:8: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  production_model = client.get_latest_versions(model_name, stages=["Production"])


In [60]:
# Make predictions
y_pred_staging = staging_model.predict(X_test)
rmse_staging = np.sqrt(mean_squared_error(df["trip_distance"], y_pred_staging))

if production_model:
    y_pred_production = production_model.predict(X_test)
    rmse_production = np.sqrt(mean_squared_error(df["trip_distance"], y_pred_production))
else:
    rmse_production = float("inf")  # No production model, so staging is automatically better

print(f"Staging RMSE: {rmse_staging:.4f}")
print(f"Production RMSE: {rmse_production:.4f}")

Staging RMSE: 10.7589
Production RMSE: 10.7589


In [61]:
staging_version = client.get_latest_versions(model_name, stages=["Staging"])[0].version

if rmse_staging < rmse_production:
    print("Updating Production model...")
    client.transition_model_version_stage(
        name=model_name,
        version=staging_version,
        stage="Production"
    )
    print(f"Model version {staging_version} promoted to Production!")
else:
    print("No update needed. Production model performs better.")

No update needed. Production model performs better.


/var/folders/9p/zg1k26nd61j1lr7bdl1cn2100000gn/T/ipykernel_68358/1352207929.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  staging_version = client.get_latest_versions(model_name, stages=["Staging"])[0].version
